# Observability study of the linear system propofol-Remifentanil to BIS

In [1]:
# Import of the necessary libraries
import numpy as np
import casadi as cas
import control
import python_anesthesia_simulator as pas
import scipy.linalg as la

## System description

Linearization of the PK-PD system for Propofol and Remifentanil effect on BIS.
$$
\left\{ 
    \begin{array}{ll}
    \dot{x} &= A x + B u \\
    y &= C x
    \end{array}
    \right.
$$

Different choices for the output matrix are possible: 
1) A linear approximation of the surface response;
2) The inverse of the half effect concentration to get the intermdiate variable $U$ as output;
3) A matrix with integer which give a rank of 7 for the observability matrix.

In [20]:
age = 50
height = 170
weight = 70
gender = 0

simulator = pas.Patient([age, height, weight, gender])

# %% get matrices of PK system
A_p = simulator.propo_pk.continuous_sys.A[:4, :4]
A_r = simulator.remi_pk.continuous_sys.A[:4, :4]
B_p = simulator.propo_pk.continuous_sys.B[:4]
B_r = simulator.remi_pk.continuous_sys.B[:4]

A = np.block([[A_p, np.zeros((4, 4))], [np.zeros((4, 4)), A_r]]) * 60 *60
B = np.block([[B_p, np.zeros((4, 1))], [np.zeros((4, 1)), B_r]])

# output matrix choice
output = 2
if output == 1:
    x = cas.MX.sym('x', 8)
    BIS = simulator.bis_pd.compute_bis(x[3], x[7])

    grad = cas.gradient(BIS, x)
    grad = cas.Function('grad', [x], [grad])

    # get the equilibrium concentration
    ratio = 2
    up, ur = simulator.find_bis_equilibrium_with_ratio(50, ratio)
    xp = up * control.dcgain(simulator.propo_pk.continuous_sys)
    xr = ur * control.dcgain(simulator.remi_pk.continuous_sys)

    C = np.array(grad([xp]*4+[xr]*4)).reshape(1, 8)
# C = np.array([[0, 0, 0, 0, 0, 0, 0, 1],
#               [0, 0, 0, 1, 0, 0, 0, 0]])
elif output == 2:
    C = np.array([[0, 0, 0, 1/simulator.bis_pd.c50p, 0, 0, 0, 1/simulator.bis_pd.c50r]])
elif output == 3:
    C = np.array([[0, 0, 0, 0.2, 0, 0, 0, 0.3]])


## Computation of the observability matrix

In [24]:
obsv = control.obsv(A, C)
print(f"Rank of the observability matrix: {np.linalg.matrix_rank(obsv)}")

Rank of the observability matrix: 7


The rank of the matrix is only 7, but all the eigenvalue of $A$ are strictly negative, 7 modes of the system are observable and 1 is detectable.

## Separation of the different mode

Let's define $T = ( K_{O}^\perp, K_{O})$, where $K_{O}$ is an orthonormal basis of the kernel of the observability matrix and $K_{O}^\perp$ an orthonormal basis of it's complementary in $\mathbb{R}^8$. 

Then let's denote $\xi = T^{-1}x$ the state in the new basis. It comes:
$$
\left\{ 
    \begin{array}{ll}
    \dot{\xi} &= T^{-1} A T \xi + T^{-1}B u \\
    y &= C T \xi
    \end{array}
    \right.
$$

Where the 6 first element of $\xi$ are the observable modes and the two last ones are only detectable.

In [26]:
# %% get the kernel of the observability matrix
kernel = la.null_space(obsv)
ortho_obs = la.orth(obsv)


# change base matrix
T = np.block([[ortho_obs, kernel]])
T.T @ T
# change base of the system
A_xi = T.T @ A @ T
B_xi = T.T @ B
C_xi = C @ T
C_xi[:, 7:] = 0

print(A_xi)
print(C_xi)

obsv_xi = control.obsv(A_xi, C_xi)
print(f"Rank of the observability matrix: {np.linalg.matrix_rank(obsv_xi)}")

[[-3.14918516e+01  4.44636008e-01  2.32868694e+00 -3.12377316e+01
   3.13316774e+00  6.59499421e-01 -9.50365641e-02 -1.25859200e-02]
 [ 5.04520197e-01 -7.52060520e-01  5.75924012e-01  5.61693523e-01
  -5.58666175e-02 -1.15087446e-02  3.55944741e-03  6.92417652e-01]
 [ 1.91073581e-02  6.21677416e-01 -1.52313416e+01  1.53105976e+01
  -1.48702857e+00 -3.25099329e-01 -5.66582665e-02  5.85904716e-02]
 [ 8.19767401e-03 -3.34478744e-01  2.60007904e+01 -5.19475781e+01
   2.57258331e+00  8.42916458e-01  2.68429864e+00  2.56436799e-02]
 [-7.83549673e-04  3.39664407e-02 -2.55758321e+00  2.56674116e+00
  -2.58188382e+01 -2.31164176e+00  2.76206120e+01  1.23690536e-03]
 [-1.64431858e-04  7.08057123e-03 -5.33024353e-01  3.67114269e-01
  -7.01551357e+00 -3.91923360e+00 -4.51071360e+00 -1.36181694e-03]
 [ 2.36498106e-05 -1.04174627e-03  7.82099290e-02 -1.10860890e-01
  -1.79463498e-01 -9.88459681e+00 -6.05708843e+01  2.71328548e-03]
 [-9.29714540e-01  7.17810343e-01 -2.20239761e-01 -4.37094388e-01
   

### Separation of the observable system

In [13]:
A_control = A_xi[:7, :7]
C_control = C_xi[:, :7]
obsv_control = control.obsv(A_control, C_control)
# print(np.linalg.cond(obsv_control))
print(f"Rank of the observability matrix: {np.linalg.matrix_rank(obsv_control)}")
# kernel = la.null_space(obsv_control)
# print(kernel)

Rank of the observability matrix: 5


After selecting the the observable part of the system the rank of the observability decrease, is this normal ?

Creation of a gain for a linear observer:

In [14]:

L = control.place(A_control.T, C_control.T, [-1, -2, -3, -4, -5, -6, -7]).T

A_control = A_control- L @ C_control

A_correct = A_xi - np.block([[L],[np.zeros(1)],[np.zeros(1)]]) @ C_xi
# eigenvalues of the system
print(f"Eigenvalues of the observable system: \n {np.linalg.eigvals(A_control)} \n ")
print(f"Eigenvalues of the full observed system: \n {np.linalg.eigvals(A_correct)} \n ")


Eigenvalues of the observable system: 
 [-6.07550256e+00+1.32956252e+05j -6.07550256e+00-1.32956252e+05j
 -7.53636829e-03+2.43628116e-02j -7.53636829e-03-2.43628116e-02j
 -1.56509257e-03+0.00000000e+00j  1.60654873e-03+0.00000000e+00j] 
 
Eigenvalues of the full observed system: 
 [-6.19505949e+00+132956.26694459j -6.19505949e+00-132956.26694459j
  2.27964280e-01     +0.j         -1.18939995e-02     +0.j
  3.35016255e-03     +0.j          1.36817980e-03     +0.j
 -9.08831306e-04     +0.j         -2.14403626e-04     +0.j        ] 
 
